<a href="https://colab.research.google.com/github/zzhining/crawling/blob/main/paxnet_stock_report_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paxnet의 증권사리포트의 종목리포트 를 수집하여 데이터 프레임으로 생성하고, 데이터프레임을 report.csv 로 저장하시오.

In [66]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "http://www.paxnet.co.kr/stock/report/report?wlog_rpt=jm&menuCode=2222"
response = requests.get(url)
html = BeautifulSoup(response.content)
div = html.find("div", {"class":"board-type"})
li_list = div.find_all("li")

#### 수집 대상 행 결정

In [67]:
print(li_list[0])

<li class="board-list-th">
<div>종목</div>
<div>제목</div>
<div class="right">적정가격</div>
<div>투자의견</div>
<div>제공출처</div>
<div class="cent">작성일</div>
</li>


In [68]:
print(li_list[1])

<li>
<div><strong class="color-cate"><span class="bracket">[</span><a href="http://www.paxnet.co.kr/stock/analysis/main?wlog_rpt=list_jm&amp;abbrSymbol=065680">우주일렉트로</a><span class="bracket">]</span></strong></div>
<div>
<p>
<a href="javascript:selectView('145144');">현금성 자산이 풍부한 초정밀 커넥터 대표 기업	</a>
</p>
</div>
<div class="line3">
<span>적정가격</span>
	 								
										
										
											-원
										
									
								</div>
<div class="line3 color-black">
									-
								</div>
<div class="line3">한국IR협의회</div>
<div class="line3">2024.06.28</div>
</li>


In [69]:
print(li_list[-1])

<li><a href="#" onclick="linkPage(5); return false;">5</a></li>


In [70]:
check_str = '<strong class="color-cate"><span class="bracket">'

In [71]:
print(check_str in str(li_list[1]))
print(check_str in str(li_list[0]))
print(check_str in str(li_list[-1]))

True
False
False


#### 파싱 전략 수립

In [72]:
stock = li_list[1].find_all("a")[0].text
stock

'우주일렉트로'

In [73]:
title = li_list[1].find_all("a")[1].text
title

'현금성 자산이 풍부한 초정밀 커넥터 대표 기업\t'

In [74]:
li_list[1].find_all("div", {"class":"line3"})

[<div class="line3">
 <span>적정가격</span>
 	 								
 										
 										
 											-원
 										
 									
 								</div>,
 <div class="line3 color-black">
 									-
 								</div>,
 <div class="line3">한국IR협의회</div>,
 <div class="line3">2024.06.28</div>]

In [75]:
price = li_list[1].find_all("div", {"class":"line3"})[0].text
price

'\n적정가격\r\n\t \t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t-원\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t'

In [76]:
price = price.replace('적정가격', '').strip()
price

'-원'

In [77]:
opinion = li_list[1].find_all("div", {"class":"line3"})[1].text.strip()
opinion

'-'

In [78]:
trading_firm = li_list[1].find_all("div", {"class":"line3"})[2].text
trading_firm

'한국IR협의회'

In [79]:
date = li_list[1].find_all("div", {"class":"line3"})[3].text
display(date)

'2024.06.28'

#### 단일 리포트 파싱

In [80]:
def parsing_report(li):
    stock = li.find_all("a")[0].text # 종목
    title = li.find_all("a")[1].text.strip() # 제목
    price = li.find_all("div", {"class":"line3"})[0].text.replace('적정가격', '').strip() # 적정가격
    opinion = li.find_all("div", {"class":"line3"})[1].text.strip() # 투자의견
    trading_firm = li.find_all("div", attrs = {"class":"line3"})[2].text # 제공출처
    date = li.find_all("div", attrs = {"class":"line3"})[3].text # 날짜
    date = pd.to_datetime(date)

    return [stock, title, price, opinion, trading_firm, date]

In [81]:
print(parsing_report(li_list[1]))

['우주일렉트로', '현금성 자산이 풍부한 초정밀 커넥터 대표 기업', '-원', '-', '한국IR협의회', Timestamp('2024-06-28 00:00:00')]


#### 한 페이지 내 모든 리포트 수집 및 정제

In [82]:
data = []
for li in li_list:
    if check_str in str(li):
        record = parsing_report(li)
        data.append(record)
df = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])
df

,종목명,리포트제목,적정가격,의견,증권사,날짜
0,우주일렉트로,현금성 자산이 풍부한 초정밀 커넥터 대표 기업,-원,-,한국IR협의회,2024-06-28
1,애니젠,펩타이드 의약품은 애니젠으로부터 시작한다,-원,-,한국IR협의회,2024-06-28
2,아이텍,다시 본업으로 집중,-원,-,한국IR협의회,2024-06-28
3,LB인베스트먼트,IT 투자에 특화된 벤처캐피탈로 금리 인하가 반갑다,-원,-,한국IR협의회,2024-06-28
4,영림원소프트랩,중소기업과 함께 성장한 토종 ERP 솔루션 기업,-원,-,한국IR협의회,2024-06-28
5,다보링크,최대주주 변경 후 사업 안정화가 관건,-원,-,유안타증권,2024-06-28
6,삼성E&A,나무가 아닌 숲을 볼 때,38000원,매수,유안타증권,2024-06-28
7,아바텍,NDR Review: 재도약의 원년,-원,매수,메리츠증권,2024-06-28
8,아모레퍼시픽,아쉬운 중국 리스크,220000원,매수,키움증권,2024-06-28
9,삼성전자,"실적 호조 속, AI 모멘텀 확대 예상",110000원,매수,키움증권,2024-06-28


In [83]:
import time

base_url = "http://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={}&reportId=0&searchKey=stock&searchValue="
data = []

# 10 페이지 데이터 수집
for page_no in range(1, 10):
    # 웹 페이지 소스 가져오기
    url = base_url.format(page_no)

    response = requests.get(url)
    # while True:
    #     response = requests.get(url)
    #     if response.status_code == 200:
    #         time.sleep(1)
    #         break
    #     else:
    #         time.sleep(10 * 60)

    html = BeautifulSoup(response.content)

    # 웹 페이지 소스에서 필요한 부분만 가져오기
    div = html.find("div", {"class":"board-type"})
    li_list = div.find_all("li")
    for li in li_list:
        if check_str in str(li):
            record = parsing_report(li)
            data.append(record)

df = pd.DataFrame(data, columns = ["종목", "제목", "적정가격", "투자의견", "제공출처", "날짜"])

In [84]:
df.shape

(270, 6)

In [85]:
df.head()

,종목,제목,적정가격,투자의견,제공출처,날짜
0,우주일렉트로,현금성 자산이 풍부한 초정밀 커넥터 대표 기업,-원,-,한국IR협의회,2024-06-28
1,애니젠,펩타이드 의약품은 애니젠으로부터 시작한다,-원,-,한국IR협의회,2024-06-28
2,아이텍,다시 본업으로 집중,-원,-,한국IR협의회,2024-06-28
3,LB인베스트먼트,IT 투자에 특화된 벤처캐피탈로 금리 인하가 반갑다,-원,-,한국IR협의회,2024-06-28
4,영림원소프트랩,중소기업과 함께 성장한 토종 ERP 솔루션 기업,-원,-,한국IR협의회,2024-06-28


In [86]:
df.tail()

,종목,제목,적정가격,투자의견,제공출처,날짜
265,LG헬로비전,본업이 개선되어야 하는데,"4,000원",매수,대신증권,2024-02-13
266,콘텐트리중앙,삼천만의 성과,"21,000원",매수,대신증권,2024-02-13
267,CJ ENM,언제 사야 돼? 지금!,"110,000원",매수,대신증권,2024-02-13
268,LG유플러스,성장 보다는 효율화의 길로,-원,-,대신증권,2024-02-13
269,DL,반가운 서프라이즈,"65,000원",매수,대신증권,2024-02-13


In [87]:
df.to_csv("report.csv", encoding = 'cp949')